<a href="https://colab.research.google.com/github/mehta-aditya13191/Machine-Learning/blob/main/score_me_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install pdfplumber pandas openpyxl numpy

import io
import re
import pandas as pd
import pdfplumber
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import base64
from typing import List, Tuple, Dict, Optional, Union
from google.colab import files

def extract_tables_from_pdf(pdf_file: Union[str, io.BytesIO], output_excel_path: Optional[str] = None) -> Tuple[List[pd.DataFrame], bool]:
    """
    Extract tables from a PDF file and optionally save them to an Excel file.

    Args:
        pdf_file: File path or file-like object containing the PDF
        output_excel_path: Optional path to save extracted tables as Excel

    Returns:
        Tuple containing:
        - List of extracted DataFrames (one per table)
        - Boolean indicating success status
    """
    print("Processing PDF...")
    extracted_dfs = []

    try:
        with pdfplumber.open(pdf_file) as pdf:
            all_tables = []

            for page in pdf.pages:
                tables = extract_page_tables(page)
                if tables:
                    all_tables.extend(tables)

            if all_tables:
                extracted_dfs = process_extracted_tables(all_tables, output_excel_path)
                return extracted_dfs, True

            return [], False

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return [], False

def extract_page_tables(page) -> List[List[List[str]]]:
    """Extract all tables from a single PDF page using multiple methods"""
    tables = []

    # Method 1: Built-in table extraction
    builtin_tables = page.extract_tables()
    if builtin_tables:
        tables.extend(builtin_tables)

    # Method 2: Bank statement extraction
    bank_table = extract_bank_statement_table(page)
    if bank_table:
        tables.append(bank_table)

    # Method 3: Text-based table extraction
    text_table = extract_text_table(page)
    if text_table:
        tables.append(text_table)

    return tables

def extract_bank_statement_table(page) -> Optional[List[List[str]]]:
    """Specialized extraction for bank statement formats"""
    text = page.extract_text()
    if not text:
        return None

    lines = text.split('\n')
    table_rows = []

    # Header row
    header = ["Date", "Description", "Amount", "Balance"]
    table_rows.append(header)

    # Transaction patterns
    date_pattern = re.compile(r'(\d{2}-[A-Za-z]{3}-\d{4})')
    amount_pattern = re.compile(r'(\d{1,3}(?:,\d{3})*\.\d{2})')

    for line in lines:
        if not line.strip() or any(x in line for x in ["BANK NAME", "Page No", "REPORT PRINTED"]):
            continue

        date_match = date_pattern.search(line)
        if not date_match:
            continue

        date = date_match.group(1)
        desc_start = date_match.end()

        # Find amounts in the line
        amounts = amount_pattern.findall(line)
        if not amounts:
            table_rows.append([date, line[desc_start:].strip(), "", ""])
            continue

        # Get description (between date and first amount)
        desc_end = line.find(amounts[0], desc_start)
        description = line[desc_start:desc_end].strip()

        # Determine if debit or credit
        if len(amounts) == 1:
            table_rows.append([date, description, amounts[0], ""])
        else:
            table_rows.append([date, description, amounts[0], amounts[1]])

    return table_rows if len(table_rows) > 1 else None

def extract_text_table(page) -> Optional[List[List[str]]]:
    """Extract tabular data from text using spacing patterns"""
    text = page.extract_text()
    if not text:
        return None

    lines = [line for line in text.split('\n') if line.strip()]
    if len(lines) < 2:
        return None

    # Find column boundaries by analyzing whitespace patterns
    boundaries = detect_column_boundaries(lines)
    if not boundaries:
        return None

    table = []
    for line in lines:
        row = []
        prev_boundary = 0
        for boundary in boundaries:
            cell = line[prev_boundary:boundary].strip()
            row.append(cell)
            prev_boundary = boundary
        # Add remaining text
        cell = line[prev_boundary:].strip()
        row.append(cell)
        table.append(row)

    return table

def detect_column_boundaries(lines: List[str]) -> List[int]:
    """Detect column boundaries by analyzing whitespace patterns"""
    whitespace_counts = []

    for line in lines[:min(20, len(lines))]:  # Analyze first 20 lines
        counts = []
        in_space = False
        space_start = 0

        for i, char in enumerate(line):
            if char.isspace():
                if not in_space:
                    space_start = i
                    in_space = True
            else:
                if in_space:
                    counts.append((space_start, i))
                    in_space = False

        whitespace_counts.append(counts)

    if not whitespace_counts:
        return []

    # Find consistent column boundaries
    boundary_candidates = {}
    for counts in whitespace_counts:
        for start, end in counts:
            if end not in boundary_candidates:
                boundary_candidates[end] = 1
            else:
                boundary_candidates[end] += 1

    # Only keep boundaries that appear in most lines
    threshold = len(whitespace_counts) * 0.7  # 70% of lines
    boundaries = [pos for pos, count in boundary_candidates.items() if count >= threshold]

    return sorted(boundaries)

def process_extracted_tables(tables: List[List[List[str]]], output_path: Optional[str]) -> List[pd.DataFrame]:
    """Convert raw tables to cleaned DataFrames and optionally save to Excel"""
    dfs = []

    if output_path:
        wb = Workbook()
        wb.remove(wb.active)  # Remove default sheet

    for i, table in enumerate(tables):
        df = clean_table(pd.DataFrame(table))
        dfs.append(df)

        if output_path:
            ws = wb.create_sheet(f"Table_{i+1}")
            for row in dataframe_to_rows(df, index=False, header=True):
                ws.append(row)

    if output_path:
        wb.save(output_path)

    return dfs

def clean_table(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and standardize extracted table"""
    # Remove empty rows and columns
    df = df.replace('', np.nan)
    df = df.dropna(how='all').dropna(axis=1, how='all')
    df = df.fillna('')

    # Set headers if first row looks like column names
    if len(df) > 1 and all(isinstance(x, str) for x in df.iloc[0]):
        df.columns = df.iloc[0]
        df = df[1:].reset_index(drop=True)

    return df

def get_table_download_link(df: pd.DataFrame, filename: str) -> str:
    """Generate download link for a DataFrame"""
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Table')
    b64 = base64.b64encode(output.getvalue()).decode()
    return f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="{filename}">Download Excel</a>'

def main():
    print("PDF Table Extractor for Google Colab")
    print("-----------------------------------")

    # Upload PDF file
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded. Please try again.")
        return

    pdf_filename = next(iter(uploaded))
    pdf_bytes = io.BytesIO(uploaded[pdf_filename])

    print(f"\nProcessing {pdf_filename}...")
    dfs, success = extract_tables_from_pdf(pdf_bytes)

    if success:
        print(f"\nSuccessfully extracted {len(dfs)} tables:")

        # Display each table
        for i, df in enumerate(dfs):
            print(f"\nTable {i+1}:")
            print(df)

            # Create download link
            excel_filename = f"{pdf_filename.split('.')[0]}_table_{i+1}.xlsx"
            print(f"\nDownload link for Table {i+1}:")
            print(get_table_download_link(df, excel_filename))

        # Combined download
        if len(dfs) > 1:
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='openpyxl') as writer:
                for i, df in enumerate(dfs):
                    df.to_excel(writer, sheet_name=f"Table_{i+1}", index=False)
            b64 = base64.b64encode(output.getvalue()).decode()
            combined_filename = f"{pdf_filename.split('.')[0]}_all_tables.xlsx"
            print(f"\nDownload all tables:")
            print(f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="{combined_filename}">Download All Tables</a>')
    else:
        print("\nNo tables found in the PDF document.")

if __name__ == "__main__":
    main()

PDF Table Extractor for Google Colab
-----------------------------------


Saving test3.pdf to test3.pdf

Processing test3.pdf...
Processing PDF...



Successfully extracted 5 tables:

Table 1:
0          Date                                        Description Amount  \
0   01-APR-2022                                                B/F          
1   04-Apr-2022                                T BY 06971000010040          
2   04-Apr-2022                                          C By Cash          
3   20-Apr-2022                                T BY 06971000010040          
4   29-Apr-2022                                          C By Cash          
5   30-Apr-2022  T 06971300000866:Int.Coll:01-04-2022 to 30-04-...          
6   30-May-2022                        T Inspection Charges Yearly          
7   31-May-2022  T 06971300000866:Int.Coll:01-05-2022 to 31-05-...          
8   30-Jun-2022  T 06971300000866:Int.Coll:01-06-2022 to 30-06-...          
9   01-Jul-2022  T Cr-IMPS :P2A/05CREDIT/IMPS/21821/9308141/jai...          
10  02-Jul-2022                         T Cr-IMPS :P2A/TRAMO TECH/          
11  02-Jul-2022                 